# <font color="red">Solar Energy Prediction with Ridge Regressor</font>

# Library Import

In [1]:
# Data manipulation and visualization
import pandas as pd
import math
import json

# Preprocessing
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge as RidgeR

import sys
sys.path.append('../../') # Uncomment this line if running locally
# sys.path.append('/kaggle/input/weatherdata') # Uncomment this line if running on Kaggle
from historyManagement import *

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Datasets Import

In [2]:
# Load the weather dataset for visibility prediction
weather_data = pd.read_csv('../../FinalDatasets/finalDataset.csv') # Uncomment this line if running locally
# weather_data = pd.read_csv('/kaggle/input/weatherdata/finalDataset.csv') # Uncomment this line if running on Kaggle
weather_data['DateTime'] = pd.to_datetime(weather_data['DateTime'])
weather_data.head()

,DateTime,Year,Month,Day,Hour,DaySegments,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,...,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy,Conditions,Icon
0,2023-01-01 00:00:00,2023,1,1,0,Late Night,0,0,0,1,...,0.49,1018.68,0.03,0.0,10.0,2.76,0.0,0.0,Clear,clear-night
1,2023-01-01 01:00:00,2023,1,1,1,Late Night,0,0,0,1,...,0.54,1018.03,0.11,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
2,2023-01-01 02:00:00,2023,1,1,2,Late Night,0,0,0,1,...,30.51,1017.56,0.03,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
3,2023-01-01 03:00:00,2023,1,1,3,Late Night,0,0,0,1,...,49.23,1018.05,0.00,0.0,10.0,2.28,0.0,0.0,Clear,clear-night
4,2023-01-01 04:00:00,2023,1,1,4,Late Night,0,0,0,1,...,49.90,1018.00,86.17,0.0,10.0,1.27,0.0,0.0,Partially cloudy,fog


In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

In [4]:
weather_data.describe()

,DateTime,Year,Month,Day,Hour,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,DaySegments_Midday,...,Windgust,WindSpeed,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy
count,14616,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,...,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000
mean,2023-11-01 11:30:00,2023.400657,5.720854,15.735632,11.500000,0.131294,0.060550,0.098043,0.249726,0.116585,...,17.068560,8.090783,159.739178,1008.265012,53.579923,2.263555,16.504787,4.368134,227.640683,0.819217
min,2023-01-01 00:00:00,2023.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.660000,0.000000,0.000000,982.420000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,2023-06-02 05:45:00,2023.000000,3.000000,8.000000,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.170000,2.870000,80.965000,1003.277500,26.670000,0.000000,10.000000,3.990000,0.000000,0.000000
50%,2023-11-01 11:30:00,2023.000000,6.000000,16.000000,11.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,14.645000,7.670000,163.790000,1007.970000,51.540000,0.000000,10.000000,4.650000,10.665000,0.010000
75%,2024-04-01 17:15:00,2024.000000,8.000000,23.000000,17.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.410000,11.750000,241.402500,1013.400000,87.790000,4.740000,12.580000,4.650000,459.745000,1.660000
max,2024-08-31 23:00:00,2024.000000,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,84.120000,71.970000,360.000000,1022.060000,100.000000,10.000000,97.740000,24.100000,1026.650000,3.700000
std,NaN,0.490048,3.195073,8.803610,6.922423,0.337734,0.238511,0.297383,0.432869,0.320936,...,10.195911,6.250203,104.422027,6.010683,35.166637,3.083074,15.031827,1.376994,307.219387,1.106444


# Data Split

In [5]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for train_idx, test_idx in sss.split(weather_data, weather_data['DaySegments']):
    weather_data_train = weather_data.iloc[train_idx]
    weather_data_test = weather_data.iloc[test_idx]

In [6]:
# Show the count of each unique class in DaySegments
class_counts = weather_data['DaySegments'].value_counts().reset_index()
class_counts.columns = ['DaySegments', 'Count']

print("Distribution of data based on DaySegments:")
print(class_counts)

Distribution of data based on DaySegments:
     DaySegments  Count
0     Late Night   3650
1        Morning   3504
2      Afternoon   1919
3         Midday   1704
4          Night   1521
5        Evening   1433
6  Early Morning    885


In [7]:
print("Distribution of DaySegments in Training Set:")
print(weather_data_train['DaySegments'].value_counts())

print("\nDistribution of DaySegments in Test Set:")
print(weather_data_test['DaySegments'].value_counts())

Distribution of DaySegments in Training Set:
DaySegments
Late Night       2920
Morning          2803
Afternoon        1535
Midday           1363
Night            1217
Evening          1146
Early Morning     708
Name: count, dtype: int64

Distribution of DaySegments in Test Set:
DaySegments
Late Night       730
Morning          701
Afternoon        384
Midday           341
Night            304
Evening          287
Early Morning    177
Name: count, dtype: int64


# Some Reused Parameters

In [8]:
grid_param = {
    'alpha': [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0],
    'tol': [1e-6, 1e-5, 1e-4, 1e-3],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'fit_intercept': [True, False],
    'positive': [False, True]
}

In [9]:
number_of_model_for_randomized_grid = 500
model_name_for_saving = "RidgeR"
target_name_for_saving = "SolarEnergy"

# All Features

In [10]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_train = weather_data_train['SolarEnergy']

In [11]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_test = weather_data_test['SolarEnergy']

In [12]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [13]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


## MinMax Scaler

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [15]:
X_train

array([[0.72727273, 0.83333333, 0.43478261, ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.26666667, 0.69565217, ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.43333333, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.86666667, 0.43478261, ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.4       , 0.91304348, ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.26666667, 0.34782609, ..., 0.8925    , 0.29      ,
        0.27812962]], shape=(11692, 27))

In [16]:
X_test

array([[0.63636364, 0.06666667, 0.95652174, ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.26666667, 0.65217391, ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.03333333, 0.2173913 , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.2       , 0.73913043, ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.13333333, 0.86956522, ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.73333333, 0.95652174, ..., 0.5161    , 0.        ,
        0.07388643]], shape=(2924, 27))

### Default Parameters

In [17]:
training_model = RidgeR(alpha=1.0, random_state=42, max_iter=10000)
training_model

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [18]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 5.47 ms


,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [19]:
y_pred = training_model.predict(X_test)
y_pred

array([-0.00350206,  0.27131455, -0.00603715, ...,  0.75227548,
        0.00647584, -0.00146537], shape=(2924,))

In [20]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [21]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [22]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.004413008374186058
MAE Score: 0.043499827777576994
R2 Score: 0.9963780199849713
RMSE Score: 0.06643047775069857


In [23]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004413, R²: 0.996378
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [24]:
grid_search = GridSearchCV(
    RidgeR(random_state=42, max_iter=10000), 
    grid_param, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [25]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
CPU times: total: 5.31 s
Wall time: 58.7 s


,estimator,Ridge(max_ite...ndom_state=42)
,param_grid,"{'alpha': [1e-05, 5e-05, ...], 'fit_intercept': [True, False], 'positive': [False, True], 'solver': ['auto', 'svd', ...], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,0.05


In [26]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'alpha': 0.05, 'fit_intercept': True, 'positive': True, 'solver': 'auto', 'tol': 1e-05}
Best Cross-Validation Score (Negative MSE): -0.004305767892372566
Best Cross-Validation Score (MSE): 0.004305767892372566
Best Cross-Validation Score (RMSE): 0.06561835027164707


In [27]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [28]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [29]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004412405409599089
MAE Score: 0.043339245066954905
R2 Score: 0.9963785148686196
RMSE Score: 0.06642593928277635


In [30]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004412, R²: 0.996379
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Standard Scaler

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [32]:
X_train

array([[ 1.01884116,  1.17229173, -0.21938027, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461, -0.75947244,  0.64624572, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -0.1913065 , -1.66209024, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943,  1.28592492, -0.21938027, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264, -0.30493969,  1.36760071, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.75947244, -0.50792226, ...,  1.01722469,
         0.20430944,  0.85928021]], shape=(11692, 27))

In [33]:
X_test

array([[ 0.70637943, -1.44127155,  1.51187171, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598, -0.75947244,  0.50197472, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -1.55490474, -0.94073525, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092, -0.98673881,  0.79051672, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747, -1.21400518,  1.22332971, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  0.83139217,  1.51187171, ..., -0.05293696,
        -0.73592134, -0.43343649]], shape=(2924, 27))

### Default Parameters

In [34]:
training_model = RidgeR(alpha=1.0, random_state=42, max_iter=10000)
training_model

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [35]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 6.72 ms


,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [36]:
y_pred = training_model.predict(X_test)
y_pred

array([-0.00336606,  0.27094282, -0.00560612, ...,  0.74929667,
        0.00568373, -0.00147245], shape=(2924,))

In [37]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [38]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [39]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.00440850089644773
MAE Score: 0.043250657506437855
R2 Score: 0.9963817195007896
RMSE Score: 0.06639654280493625


In [40]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004409, R²: 0.996382
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [41]:
grid_search = GridSearchCV(
    RidgeR(random_state=42, max_iter=10000), 
    grid_param, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [42]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
CPU times: total: 7.2 s
Wall time: 2min 8s


,estimator,Ridge(max_ite...ndom_state=42)
,param_grid,"{'alpha': [1e-05, 5e-05, ...], 'fit_intercept': [True, False], 'positive': [False, True], 'solver': ['auto', 'svd', ...], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,1.0


In [43]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'alpha': 1.0, 'fit_intercept': True, 'positive': True, 'solver': 'auto', 'tol': 0.001}
Best Cross-Validation Score (Negative MSE): -0.00430670302006423
Best Cross-Validation Score (MSE): 0.00430670302006423
Best Cross-Validation Score (RMSE): 0.0656254753892437


In [44]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [45]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [46]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004412099978272335
MAE Score: 0.043494331169052804
R2 Score: 0.9963787655516158
RMSE Score: 0.06642364020642301


In [47]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004412, R²: 0.996379
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Robust Scaler

In [48]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [49]:
X_train

array([[ 0.6       ,  0.66666667, -0.16666667, ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        , -0.46666667,  0.33333333, ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -0.13333333, -1.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       ,  0.73333333, -0.16666667, ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        , -0.2       ,  0.75      , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.46666667, -0.33333333, ...,  0.61755563,
         0.59917355,  7.5       ]], shape=(11692, 27))

In [50]:
X_test

array([[ 4.00000000e-01, -8.66666667e-01,  8.33333333e-01, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -4.66666667e-01,  2.50000000e-01, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -9.33333333e-01, -5.83333333e-01, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01, -6.00000000e-01,  4.16666667e-01, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01, -7.33333333e-01,  6.66666667e-01, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  4.66666667e-01,  8.33333333e-01, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]],
      shape=(2924, 27))

### Default Parameters

In [51]:
training_model = RidgeR(alpha=1.0, random_state=42, max_iter=10000)
training_model

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [52]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 31.2 ms
Wall time: 5.12 ms


,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [53]:
y_pred = training_model.predict(X_test)
y_pred

array([-0.00342993,  0.2710589 , -0.00574567, ...,  0.74969351,
        0.0057469 , -0.00150008], shape=(2924,))

In [54]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [55]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [56]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.004408815884172687
MAE Score: 0.043285998524124424
R2 Score: 0.9963814609743723
RMSE Score: 0.06639891478158877


In [57]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004409, R²: 0.996381
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [58]:
grid_search = GridSearchCV(
    RidgeR(random_state=42, max_iter=1000), 
    grid_param, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [59]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
CPU times: total: 11.2 s
Wall time: 18min 23s


,estimator,Ridge(max_ite...ndom_state=42)
,param_grid,"{'alpha': [1e-05, 5e-05, ...], 'fit_intercept': [True, False], 'positive': [False, True], 'solver': ['auto', 'svd', ...], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,0.05


In [60]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'alpha': 0.05, 'fit_intercept': True, 'positive': True, 'solver': 'auto', 'tol': 1e-06}
Best Cross-Validation Score (Negative MSE): -0.004305841435737137
Best Cross-Validation Score (MSE): 0.004305841435737137
Best Cross-Validation Score (RMSE): 0.06561891065643453


In [61]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [62]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [63]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004419360342539913
MAE Score: 0.04341707068080941
R2 Score: 0.9963728066020627
RMSE Score: 0.06647826970175978


In [64]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 0.004419, R²: 0.996373
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


# Selected Features

In [65]:
numeric_cols = weather_data.select_dtypes(include=[float, int]).columns
correlations = weather_data[numeric_cols].corr()['SolarEnergy'].sort_values(ascending=False)
print(correlations)

SolarEnergy                  1.000000
SolarRadiation               0.999867
UVIndex                      0.998178
DaySegments_Midday           0.594829
Temp                         0.400703
FeelsLike                    0.344667
DaySegments_Morning          0.243917
WindSpeed                    0.216874
DaySegments_Afternoon        0.216045
WindDir                      0.181173
SevereRisk                   0.125741
Hour                         0.095805
Season_Summer                0.093686
Visibility                   0.054855
SeaLevelPressure             0.026544
Year                         0.022062
Dew                          0.014013
Day                         -0.001489
PrecipProb                  -0.013347
Season_Rainy                -0.032901
Season_Autumn               -0.035246
Season_Winter               -0.035617
Precip                      -0.039120
CloudCover                  -0.055435
Month                       -0.061170
Windgust                    -0.100191
DaySegments_

In [66]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

## Run Again

In [67]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Energy
                                                'Dew', # low correlation with Solar Energy
                                                'Precip', # low correlation with Solar Energy
                                                'PrecipProb']) # low correlation with Solar Energy
y_train = weather_data_train['SolarEnergy']

In [68]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Energy
                                                'Dew', # low correlation with Solar Energy
                                                'Precip', # low correlation with Solar Energy
                                                'PrecipProb']) # low correlation with Solar Energy
y_test = weather_data_test['SolarEnergy']

In [69]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [70]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


### MinMax Scaler

In [71]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [72]:
X_train

array([[0.72727273, 0.43478261, 0.        , ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.69565217, 1.        , ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.        , 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.43478261, 0.        , ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.91304348, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.34782609, 0.        , ..., 0.8925    , 0.29      ,
        0.27812962]], shape=(11692, 22))

In [73]:
X_test

array([[0.63636364, 0.95652174, 0.        , ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.65217391, 1.        , ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.2173913 , 0.        , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.73913043, 1.        , ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.86956522, 0.        , ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.95652174, 0.        , ..., 0.5161    , 0.        ,
        0.07388643]], shape=(2924, 22))

#### Default Parameters

In [74]:
training_model = RidgeR(alpha=1.0, random_state=42, max_iter=10000)
training_model

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [75]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 3.98 ms


,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [76]:
y_pred = training_model.predict(X_test)
y_pred

array([-1.88175196e-03,  2.76493233e-01, -1.40088784e-03, ...,
        7.52311338e-01,  8.65887956e-03, -2.73959375e-04], shape=(2924,))

In [77]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [78]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [79]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.004417755559788212
MAE Score: 0.04335992121724755
R2 Score: 0.9963741237287396
RMSE Score: 0.0664661986259799


In [80]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 0.004413 ≈ 0.004418 (similar)
   R²:  0.996378 ≈ 0.996374 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [81]:
grid_search = GridSearchCV(
    RidgeR(random_state=42, max_iter=10000), 
    grid_param, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [82]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
CPU times: total: 5.38 s
Wall time: 45 s


,estimator,Ridge(max_ite...ndom_state=42)
,param_grid,"{'alpha': [1e-05, 5e-05, ...], 'fit_intercept': [True, False], 'positive': [False, True], 'solver': ['auto', 'svd', ...], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,0.1


In [83]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'alpha': 0.1, 'fit_intercept': True, 'positive': False, 'solver': 'sag', 'tol': 0.001}
Best Cross-Validation Score (Negative MSE): -0.004310879055599322
Best Cross-Validation Score (MSE): 0.004310879055599322
Best Cross-Validation Score (RMSE): 0.06565728486313854


In [84]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [85]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [86]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004421665704073789
MAE Score: 0.04303177018951672
R2 Score: 0.9963709144748119
RMSE Score: 0.0664956066524232


In [87]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 0.004412 ≈ 0.004422 (similar)
   R²:  0.996379 ≈ 0.996371 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


### Standard Scaler

In [88]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [89]:
X_train

array([[ 1.01884116, -0.21938027, -0.38875095, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461,  0.64624572,  2.57234098, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -1.66209024, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943, -0.21938027, -0.38875095, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264,  1.36760071, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.50792226, -0.38875095, ...,  1.01722469,
         0.20430944,  0.85928021]], shape=(11692, 22))

In [90]:
X_test

array([[ 0.70637943,  1.51187171, -0.38875095, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598,  0.50197472,  2.57234098, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -0.94073525, -0.38875095, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092,  0.79051672,  2.57234098, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747,  1.22332971, -0.38875095, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  1.51187171, -0.38875095, ..., -0.05293696,
        -0.73592134, -0.43343649]], shape=(2924, 22))

#### Default Parameters

In [91]:
training_model = RidgeR(alpha=1.0, random_state=42, max_iter=10000)
training_model

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [92]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 6.43 ms


,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [93]:
y_pred = training_model.predict(X_test)
y_pred

array([-1.52361779e-03,  2.75371834e-01, -9.98173495e-04, ...,
        7.49590978e-01,  8.28481144e-03, -1.55923157e-04], shape=(2924,))

In [94]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [95]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [96]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.004412463530643549
MAE Score: 0.04312648157244134
R2 Score: 0.9963784671657276
RMSE Score: 0.06642637676889768


In [97]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 0.004409 ≈ 0.004412 (similar)
   R²:  0.996382 ≈ 0.996378 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [98]:
grid_search = GridSearchCV(
    RidgeR(random_state=42, max_iter=10000), 
    grid_param, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [99]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
CPU times: total: 5.44 s
Wall time: 35.5 s


,estimator,Ridge(max_ite...ndom_state=42)
,param_grid,"{'alpha': [1e-05, 5e-05, ...], 'fit_intercept': [True, False], 'positive': [False, True], 'solver': ['auto', 'svd', ...], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,0.5


In [100]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'alpha': 0.5, 'fit_intercept': True, 'positive': False, 'solver': 'sag', 'tol': 0.001}
Best Cross-Validation Score (Negative MSE): -0.004310103129409354
Best Cross-Validation Score (MSE): 0.004310103129409354
Best Cross-Validation Score (RMSE): 0.06565137568558144


In [101]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [102]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [103]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004414189024552119
MAE Score: 0.043179953836899
R2 Score: 0.996377050965276
RMSE Score: 0.06643936351706056


In [104]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 0.004412 ≈ 0.004414 (similar)
   R²:  0.996379 ≈ 0.996377 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


### Robust Scaler

In [105]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [106]:
X_train

array([[ 0.6       , -0.16666667,  0.        , ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        ,  0.33333333,  1.        , ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -1.        ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       , -0.16666667,  0.        , ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        ,  0.75      ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.33333333,  0.        , ...,  0.61755563,
         0.59917355,  7.5       ]], shape=(11692, 22))

In [107]:
X_test

array([[ 4.00000000e-01,  8.33333333e-01,  0.00000000e+00, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.50000000e-01,  1.00000000e+00, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -5.83333333e-01,  0.00000000e+00, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01,  4.16666667e-01,  1.00000000e+00, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01,  6.66666667e-01,  0.00000000e+00, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  8.33333333e-01,  0.00000000e+00, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]],
      shape=(2924, 22))

#### Default Parameters

In [108]:
training_model = RidgeR(alpha=1.0, random_state=42, max_iter=10000)
training_model

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [109]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 5.29 ms


,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,10000
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [110]:
y_pred = training_model.predict(X_test)
y_pred

array([-1.56374494e-03,  2.75571437e-01, -1.05391120e-03, ...,
        7.49976891e-01,  8.35787031e-03, -1.54792140e-04], shape=(2924,))

In [111]:
y_test

5158     0.00
3831     0.36
5117     0.00
6326     2.70
11359    0.49
         ... 
10290    0.30
13995    0.00
9665     0.90
2276     0.00
4174     0.00
Name: SolarEnergy, Length: 2924, dtype: float64

In [112]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [113]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 0.00441277257472236
MAE Score: 0.04315957862525393
R2 Score: 0.9963782135175624
RMSE Score: 0.06642870294324855


In [114]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 0.004409 ≈ 0.004413 (similar)
   R²:  0.996381 ≈ 0.996378 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [115]:
grid_search = GridSearchCV(
    RidgeR(random_state=42, max_iter=1000), 
    grid_param, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, # allow parallel processing, you cant use your own laptop while this runs
    verbose=1
)

In [116]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
CPU times: total: 5.59 s
Wall time: 1min 5s


,estimator,Ridge(max_ite...ndom_state=42)
,param_grid,"{'alpha': [1e-05, 5e-05, ...], 'fit_intercept': [True, False], 'positive': [False, True], 'solver': ['auto', 'svd', ...], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,0.1


In [117]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'alpha': 0.1, 'fit_intercept': True, 'positive': False, 'solver': 'sparse_cg', 'tol': 0.0001}
Best Cross-Validation Score (Negative MSE): -0.0043116392268425065
Best Cross-Validation Score (MSE): 0.0043116392268425065
Best Cross-Validation Score (RMSE): 0.06566307354093705


In [118]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [119]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [120]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 0.004412359719121519
MAE Score: 0.04315563249562308
R2 Score: 0.9963785523691141
RMSE Score: 0.06642559536143819


In [121]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 0.004419 → 0.004412 (lower is better)
   R²:  0.996373 → 0.996379 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


# All Performance

In [122]:
show_model_history('regression', model_name=model_name_for_saving, target_column=target_name_for_saving)


📊 REGRESSION Model Performance History
       Model         Scaler      Target  Features      MSE      MAE     RMSE       R²
RidgeR Tuned   RobustScaler SolarEnergy        22 0.004412 0.043156 0.066426 0.996379
      RidgeR StandardScaler SolarEnergy        22 0.004412 0.043126 0.066426 0.996378
      RidgeR   RobustScaler SolarEnergy        22 0.004413 0.043160 0.066429 0.996378
RidgeR Tuned StandardScaler SolarEnergy        22 0.004414 0.043180 0.066439 0.996377
      RidgeR   MinMaxScaler SolarEnergy        22 0.004418 0.043360 0.066466 0.996374
RidgeR Tuned   MinMaxScaler SolarEnergy        22 0.004422 0.043032 0.066496 0.996371

📈 Total models shown: 6
🏆 Best R² Score: 0.996379 (RidgeR Tuned + RobustScaler for SolarEnergy)


# <center><font size="50" color="red">Thank You</font></center>